# Data access and discovery

## Authors & Contributors

### Authors

- Pier Lorenzo Marasco, Ispra (Italy), [@pl-marasco](https://github.com/pl-marasco)
- Alejandro Coca-Castro, The Alan Turing Institute (United Kingdom), [@acocac](https://github.com/acocac)
- Anne Fouilloux, Simula Research Laboratory (Norway), [@annefou](https://github.com/annefou)

### Contributors
- Tina Odaka, Ifremer (France), [@tinaok](https://github.com/tinaok)

<div class="alert alert-info">
    <i class="fa-question-circle fa" style="font-size: 22px;color:#666;"></i> <b>Overview</b>
    <br>
    <br>
    <b>Questions</b>
    <ul>
        <li>How to access online (remote) datasets?</li>
        <li>How to prepare and discover online geoscience datasets?</li>
        <li>What is Analysis Ready, Cloud Optimized data (ARCO)?</li>
        <li>What is pangeo-forge?</li>
        <li>What is stac?</li>
    </ul>
    <b>Objectives</b>
    <ul>
        <li>Learn to access datasets from online object storage</li>
        <li>Learn about preparing and discovery online datasets</li>
        <li>Learn about Analysis Cloud Optimized (ARCO) data</li>
        <li>Learn about the Pangeo Forge initiative</li>
        <li>Learn about stac</li>
    </ul>
</div>

## Context

We will be using CMIP6 data and access them through [S3-compatible storage](https://en.wikipedia.org/wiki/Amazon_S3) using [Pangeo Catalog](https://pangeo-data.github.io/pangeo-cmip6-cloud/pangeo_catalog.html). 

We will also discuss the generation of Cloud-Optimised Datasets by introducing the [pangeo-forge](https://pangeo-forge.org) initiative. Finally, we will explore [Sentinel-2 Cloud-Optimised Dataset](https://registry.opendata.aws/sentinel-2-l2a-cogs/) online through SpatioTemporal Asset Catalogs ([STAC](https://stacspec.org/en)).

## Setup

This episode uses the following main Python packages:

- xarray {cite:ps}`c-xarray-hoyer2017` with [`netCDF4`](https://pypi.org/project/h5netcdf/) and [`h5netcdf`](https://pypi.org/project/h5netcdf/) engines
- intake-esm {cite:ps}`c-intake-esm`
- s3fs {cite:ps}`c-s3fs-2016`

Please install these packages if not already available in your Python environment.

### Packages

In this episode, Python packages are imported when we start to use them. However, for best software practices, we recommend you to install and import all the necessary libraries at the top of your Jupyter notebook.

### Introduction to CMIP6 data

Coupled Model Intercomparison project Phase 6
- Project under World Climate Research Programme (WCRP) 
- Since 1995 CMIP has coordinated climate model experiments 
- Defines common experiment protocols, forcings and output. 
- 33 model groups participate


#### Very useful links:
- Database for data request: http://clipc-services.ceda.ac.uk/dreq/index.html
    - **-->SEARCH FOR VARIABLES**: http://clipc-services.ceda.ac.uk/dreq/mipVars.html
    - Search for experiments: http://clipc-services.ceda.ac.uk/dreq/experiments.html
- Overview: https://www.wcrp-climate.org/wgcm-cmip/wgcm-cmip6
- ES-DOCs: https://search.es-doc.org/


#### Other links:
GMD special issue with articles explaining all MIPs in CMIP6 :
https://www.geosci-model-dev.net/special_issue590.html

- General CMIP6 website https://www.wcrp-climate.org/wgcm-cmip/wgcm-cmip6
- Guidance documents: (https://pcmdi.llnl.gov/CMIP6/)
- Emissions/Forcing datasets (https://esgf-node.llnl.gov/projects/input4mips/)
- Participating Modelling Groups (https://rawgit.com/WCRP-CMIP/CMIP6_CVs/master/src/CMIP6_institution_id.html)
- Model and experiment documentation (https://search.es-doc.org/)
- CMIP6 ESMValTool evaluation and analysis results (https://cmip-esmvaltool.dkrz.de/)

- Emission visualising: https://eccad.aeris-data.fr


#### Advantages:
- Homogenized and standardized outputs
- Same variable name
- Same experiments

#### Experiments (DECK)
![image](https://user-images.githubusercontent.com/17406708/139691209-ec237004-637b-4947-bb12-104d78a2fe44.png)



### S3-compatible Object Storage to access online data

Up to now we have downloaded data locally and then opened with Xarray `open_dataset`. When willing to manipulate large amount of data, this approach is not optimal (since it requires a lot of unnecessary local downloads). Sharing data online as Object Storage allows for data sharing and access to much larger amounts of data.

One of the most popular methods to access online remote data is through Amazon Simple Storage Service (S3) and you don't necessarily need to use Amazon services to benefit from S3 object storage. Many other providers offer S3-compatible object storage that can be accessed in a very similar way.

When using S3-compatible object storage, you still need to list all the files you would like to access. With such amount of data, it would be very cumbersome. This is why a catalog is created: this catalog is a text file (`json` format) which describes where and which data to get. It adds additional metadata too. CMIP6 Pangeo online catalog can be loaded using `intake-esm` Python package.


### Introduction to the Pangeo CMIP6 online catalog

In [1]:
import xarray as xr

xr.set_options(display_style="html")
import intake
import cftime

### Open CMIP6 online catalog

In [2]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

,unique
activity_id,18
institution_id,36
source_id,88
experiment_id,170
member_id,657
table_id,37
variable_id,700
grid_label,10
zstore,514818
dcpp_init_year,60


### Search corresponding data


In [3]:
cat = col.search(
    source_id=["CESM2"],
    experiment_id=["historical"],
    table_id=["Amon"],
    variable_id=["tas"],
    member_id=["r1i1p1f1"],
)
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,NCAR,CESM2,historical,r1i1p1f1,Amon,tas,gn,gs://cmip6/CMIP6/CMIP/NCAR/CESM2/historical/r1...,NaN,20190308


## Create dictionary from the list of datasets we found
- This step may take several minutes so be patient!

In [4]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={"use_cftime": True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [5]:
dataset_list = list(dset_dict.keys())
dataset_list

['CMIP.NCAR.CESM2.historical.Amon.gn']

## Open dataset

- Use `xarray` python package to analyze netCDF dataset
- `open_dataset` allows to get all the metadata without loading data into memory. 
- with `xarray`, we only load into memory what is needed.

In [6]:
dset = dset_dict[dataset_list[0]]
dset

<xarray.Dataset>
Dimensions:    (lat: 192, nbnd: 2, lon: 288, member_id: 1, time: 1980)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
    lat_bnds   (lat, nbnd) float32 dask.array<chunksize=(192, 2), meta=np.ndarray>
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    lon_bnds   (lon, nbnd) float32 dask.array<chunksize=(288, 2), meta=np.ndarray>
  * time       (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
    time_bnds  (time, nbnd) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
  * member_id  (member_id) <U8 'r1i1p1f1'
Dimensions without coordinates: nbnd
Data variables:
    tas        (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 600, 192, 288), meta=np.ndarray>
Attributes: (12/50)
    Conventions:             CF-1.7 CMIP-6.2
    activity_id:             CMIP
    branch_method:           standard
    branch_time_in_child:    674885.0
    branch_time_in_parent:   219000.0
    case_id:                 15
    ...                      ...
    variant_label:           r1i1p1f1
    status:                  2019-10-25;created;by nhn2@columbia.edu
    netcdf_tracking_ids:     hdl:21.14100/d9a7225a-49c3-4470-b7ab-a8180926f839
    version_id:              v20190308
    intake_esm_varname:      ['tas']
    intake_esm_dataset_key:  CMIP.NCAR.CESM2.historical.Amon.gn

### Get metadata corresponding to near-surface air temperature (tas)

In [7]:
print(dset["tas"])

<xarray.DataArray 'tas' (member_id: 1, time: 1980, lat: 192, lon: 288)>
dask.array<broadcast_to, shape=(1, 1980, 192, 288), dtype=float32, chunksize=(1, 600, 192, 288), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * time       (time) object 1850-01-15 12:00:00 ... 2014-12-15 12:00:00
  * member_id  (member_id) <U8 'r1i1p1f1'
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        near-surface (usually, 2 meter) air temperature
    description:    near-surface (usually, 2 meter) air temperature
    frequency:      mon
    id:             tas
    long_name:      Near-Surface Air Temperature
    mipTable:       Amon
    out_name:       tas
    prov:           Amon ((isd.003))
    realm:          atmos
    standard_name:  air_temperature
    time:           time
    time_label:     time-mean
 

In [8]:
dset.time.values

array([cftime.DatetimeNoLeap(1850, 1, 15, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 2, 14, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(1850, 3, 15, 12, 0, 0, 0, has_year_zero=True),
       ...,
       cftime.DatetimeNoLeap(2014, 10, 15, 12, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2014, 11, 15, 0, 0, 0, 0, has_year_zero=True),
       cftime.DatetimeNoLeap(2014, 12, 15, 12, 0, 0, 0, has_year_zero=True)],
      dtype=object)

In [9]:
dset.sel(lat=60, lon=10.75, method="nearest").sel(time="1850-12-15")["tas"].values

array([[269.032]], dtype=float32)

:::{warning}
The same dataset can be available from different locations e.g. ESGF, Zenodo, S3-compatible Object storage, etc.
How do you know if it corresponds to the very same dataset? You cannot know except if the datasets have a persistent identifier such as a Digital Object Identifier.
It is therefore recommended *1)* to be extra careful about where you get your datasets, and *2)* to double check that the content is exactly what you expect (for instance, you can perform basic quality checks).                                                            
:::

### Access remote files from S3-compatible Object Storage

The Pangeo CMIP6 online catalog is very user-friendly. However, the complete ESGF CMIP6 catalog is close to 20 PB and so far only a very tiny amount of cloud-optimised CMIP6 data has been generated (approximately 1PB).
Therefore, you may have to download CMIP6 data or other kind of data for instance when comparing CMIP6 model outputs with observations.

Rather than downloading datasets locally and individually, it is recommended to share what you download to everyone. Additional datasets are being stored and made 
publicly available in OpenStack Object storage (Swift).

In [10]:
import s3fs
import xarray as xr

In [11]:
fs = s3fs.S3FileSystem(
    anon=True, client_kwargs={"endpoint_url": "https://object-store.cloud.muni.cz"}
)

:::{tip}
The parameter `anon` is for `anonymous` and is set to `True` because the data we have stored at `https://object-store.cloud.muni.cz` is public
:::

#### List files and folders in existing buckets

Instead of organizing files in various folders, object storage systems store files in a flat organization of containers (called "buckets"). 

In [12]:
fs.ls("MODIS")

['MODIS/MOD08_M3.A2000032.061.2017276183309.nc',
 'MODIS/MOD08_M3.A2000061.061.2017272215822.nc',
 'MODIS/MOD08_M3.A2000092.061.2017276174940.nc',
 'MODIS/MOD08_M3.A2000122.061.2017275191641.nc',
 'MODIS/MOD08_M3.A2000153.061.2017276072839.nc',
 'MODIS/MOD08_M3.A2000183.061.2017276075622.nc',
 'MODIS/MOD08_M3.A2000214.061.2017276050502.nc',
 'MODIS/MOD08_M3.A2000245.061.2017276075932.nc',
 'MODIS/MOD08_M3.A2000275.061.2017276173030.nc',
 'MODIS/MOD08_M3.A2000306.061.2017276190346.nc',
 'MODIS/MOD08_M3.A2000336.061.2017276203640.nc',
 'MODIS/MOD08_M3.A2001001.061.2017277122349.nc',
 'MODIS/MOD08_M3.A2001032.061.2017277112301.nc',
 'MODIS/MOD08_M3.A2001060.061.2017277125646.nc',
 'MODIS/MOD08_M3.A2001091.061.2017277145429.nc',
 'MODIS/MOD08_M3.A2001121.061.2017277185847.nc',
 'MODIS/MOD08_M3.A2001152.061.2017277193228.nc',
 'MODIS/MOD08_M3.A2001182.061.2017277232025.nc',
 'MODIS/MOD08_M3.A2001213.061.2017278042002.nc',
 'MODIS/MOD08_M3.A2001244.061.2017278143507.nc',
 'MODIS/MOD08_M3.A20

In [13]:
fs.ls("MODIS/MOD08_M3.A2000183.061.2017276075622.nc")

['MODIS/MOD08_M3.A2000183.061.2017276075622.nc']

#### Access remote files from S3-compatible Object Storage

In [14]:
s3path = "s3://MODIS/MOD08_M3.A2000183.061.2017276075622.nc"

In [15]:
modis = xr.open_dataset(fs.open(s3path))

In [16]:
modis

<xarray.Dataset>
Dimensions:                                              (YDim:mod08: 180,
                                                          XDim:mod08: 360,
                                                          Effective_Optical_Depth_Average_Ocean_Micron_Levels:mod08: 7)
Dimensions without coordinates: YDim:mod08, XDim:mod08,
                                Effective_Optical_Depth_Average_Ocean_Micron_Levels:mod08
Data variables: (12/13)
    Aerosol_Optical_Depth_Land_Ocean_Mean_Mean           (YDim:mod08, XDim:mod08) float32 ...
    AOD_550_Dark_Target_Deep_Blue_Combined_Mean_Mean     (YDim:mod08, XDim:mod08) float32 ...
    Aerosol_Optical_Depth_Average_Ocean_Mean_Mean        (Effective_Optical_Depth_Average_Ocean_Micron_Levels:mod08, YDim:mod08, XDim:mod08) float32 ...
    Cloud_Top_Temperature_Mean_Mean                      (YDim:mod08, XDim:mod08) float32 ...
    Cloud_Effective_Emissivity_Mean_Mean                 (YDim:mod08, XDim:mod08) float32 ...
    Cloud_Fraction_Mean_Mean                             (YDim:mod08, XDim:mod08) float32 ...
    ...                                                   ...
    Cloud_Effective_Radius_Liquid_Mean_Mean              (YDim:mod08, XDim:mod08) float32 ...
    Cloud_Water_Path_Liquid_Mean_Mean                    (YDim:mod08, XDim:mod08) float32 ...
    Cloud_Water_Path_Ice_Mean_Mean                       (YDim:mod08, XDim:mod08) float32 ...
    Effective_Optical_Depth_Average_Ocean_Micron_Levels  (Effective_Optical_Depth_Average_Ocean_Micron_Levels:mod08) float32 ...
    XDim                                                 (XDim:mod08) float32 ...
    YDim                                                 (YDim:mod08) float32 ...
Attributes:
    HDFEOSVersion:                     HDFEOS_V2.19
    StructMetadata.0:                  GROUP=SwathStructure\nEND_GROUP=SwathS...
    StructMetadata.1:                  Min"\n\t\t\t\tDataType=DFNT_INT16\n\t\...
    StructMetadata.2:                  BJECT=DataField_295\n\t\t\tOBJECT=Data...
    StructMetadata.3:                  NT16\n\t\t\t\tDimList=("YDim","XDim")\...
    StructMetadata.4:                  \t\tDataFieldName="Cloud_Retrieval_Fra...
    StructMetadata.5:                  List=("YDim","XDim")\n\t\t\tEND_OBJECT...
    StructMetadata.6:                  CT=DataField_1037\n\t\t\t\tDataFieldNa...
    CoreMetadata.0:                    \nGROUP                  = INVENTORYME...
    ArchiveMetadata.0:                 \nGROUP                  = ARCHIVEDMET...
    identifier_product_doi:            10.5067/MODIS/MOD08_M3.061
    identifier_product_doi_authority:  http://dx.doi.org

### Rename dimensions and variables 

In [17]:
modis = modis.rename_dims(
    {
        "YDim:mod08": "lat",
        "XDim:mod08": "lon",
        "Effective_Optical_Depth_Average_Ocean_Micron_Levels:mod08": "levels",
    }
)

In [18]:
modis = modis.rename_vars(
    {
        "YDim": "lat",
        "XDim": "lon",
        "Effective_Optical_Depth_Average_Ocean_Micron_Levels": "levels",
    }
)

In [19]:
modis

<xarray.Dataset>
Dimensions:                                           (lat: 180, lon: 360,
                                                       levels: 7)
Dimensions without coordinates: lat, lon, levels
Data variables: (12/13)
    Aerosol_Optical_Depth_Land_Ocean_Mean_Mean        (lat, lon) float32 nan ...
    AOD_550_Dark_Target_Deep_Blue_Combined_Mean_Mean  (lat, lon) float32 nan ...
    Aerosol_Optical_Depth_Average_Ocean_Mean_Mean     (levels, lat, lon) float32 ...
    Cloud_Top_Temperature_Mean_Mean                   (lat, lon) float32 -1.4...
    Cloud_Effective_Emissivity_Mean_Mean              (lat, lon) float32 0.93...
    Cloud_Fraction_Mean_Mean                          (lat, lon) float32 0.88...
    ...                                                ...
    Cloud_Effective_Radius_Liquid_Mean_Mean           (lat, lon) float32 12.3...
    Cloud_Water_Path_Liquid_Mean_Mean                 (lat, lon) float32 146....
    Cloud_Water_Path_Ice_Mean_Mean                    (lat, lon) float32 249....
    levels                                            (levels) float32 0.47 ....
    lon                                               (lon) float32 -179.5 .....
    lat                                               (lat) float32 89.5 ... ...
Attributes:
    HDFEOSVersion:                     HDFEOS_V2.19
    StructMetadata.0:                  GROUP=SwathStructure\nEND_GROUP=SwathS...
    StructMetadata.1:                  Min"\n\t\t\t\tDataType=DFNT_INT16\n\t\...
    StructMetadata.2:                  BJECT=DataField_295\n\t\t\tOBJECT=Data...
    StructMetadata.3:                  NT16\n\t\t\t\tDimList=("YDim","XDim")\...
    StructMetadata.4:                  \t\tDataFieldName="Cloud_Retrieval_Fra...
    StructMetadata.5:                  List=("YDim","XDim")\n\t\t\tEND_OBJECT...
    StructMetadata.6:                  CT=DataField_1037\n\t\t\t\tDataFieldNa...
    CoreMetadata.0:                    \nGROUP                  = INVENTORYME...
    ArchiveMetadata.0:                 \nGROUP                  = ARCHIVEDMET...
    identifier_product_doi:            10.5067/MODIS/MOD08_M3.061
    identifier_product_doi_authority:  http://dx.doi.org

### Coordinates

In [20]:
x = modis.lon.squeeze().reset_coords(drop=True)
y = modis.lat.squeeze().reset_coords(drop=True)
z = modis.levels.squeeze().reset_coords(drop=True)

In [21]:
modis = modis.assign_coords({"lon": x, "lat": y, "levels": z})

In [22]:
modis

<xarray.Dataset>
Dimensions:                                           (lat: 180, lon: 360,
                                                       levels: 7)
Coordinates:
  * levels                                            (levels) float32 0.47 ....
  * lon                                               (lon) float32 -179.5 .....
  * lat                                               (lat) float32 89.5 ... ...
Data variables:
    Aerosol_Optical_Depth_Land_Ocean_Mean_Mean        (lat, lon) float32 nan ...
    AOD_550_Dark_Target_Deep_Blue_Combined_Mean_Mean  (lat, lon) float32 nan ...
    Aerosol_Optical_Depth_Average_Ocean_Mean_Mean     (levels, lat, lon) float32 ...
    Cloud_Top_Temperature_Mean_Mean                   (lat, lon) float32 -1.4...
    Cloud_Effective_Emissivity_Mean_Mean              (lat, lon) float32 0.93...
    Cloud_Fraction_Mean_Mean                          (lat, lon) float32 0.88...
    Cloud_Optical_Thickness_Liquid_Mean_Mean          (lat, lon) float32 20.9...
    Cloud_Effective_Radius_Liquid_Mean_Mean           (lat, lon) float32 12.3...
    Cloud_Water_Path_Liquid_Mean_Mean                 (lat, lon) float32 146....
    Cloud_Water_Path_Ice_Mean_Mean                    (lat, lon) float32 249....
Attributes:
    HDFEOSVersion:                     HDFEOS_V2.19
    StructMetadata.0:                  GROUP=SwathStructure\nEND_GROUP=SwathS...
    StructMetadata.1:                  Min"\n\t\t\t\tDataType=DFNT_INT16\n\t\...
    StructMetadata.2:                  BJECT=DataField_295\n\t\t\tOBJECT=Data...
    StructMetadata.3:                  NT16\n\t\t\t\tDimList=("YDim","XDim")\...
    StructMetadata.4:                  \t\tDataFieldName="Cloud_Retrieval_Fra...
    StructMetadata.5:                  List=("YDim","XDim")\n\t\t\tEND_OBJECT...
    StructMetadata.6:                  CT=DataField_1037\n\t\t\t\tDataFieldNa...
    CoreMetadata.0:                    \nGROUP                  = INVENTORYME...
    ArchiveMetadata.0:                 \nGROUP                  = ARCHIVEDMET...
    identifier_product_doi:            10.5067/MODIS/MOD08_M3.061
    identifier_product_doi_authority:  http://dx.doi.org

In [23]:
modis.sel(lat=60, lon=10.75, method="nearest")[
    "Aerosol_Optical_Depth_Land_Ocean_Mean_Mean"
].values

array(0.134, dtype=float32)

#### Access multiple remote files 

In [24]:
s3path = "s3://MODIS/*.nc"

In [25]:
remote_files = fs.glob(s3path)
remote_files

['MODIS/MOD08_M3.A2000032.061.2017276183309.nc',
 'MODIS/MOD08_M3.A2000061.061.2017272215822.nc',
 'MODIS/MOD08_M3.A2000092.061.2017276174940.nc',
 'MODIS/MOD08_M3.A2000122.061.2017275191641.nc',
 'MODIS/MOD08_M3.A2000153.061.2017276072839.nc',
 'MODIS/MOD08_M3.A2000183.061.2017276075622.nc',
 'MODIS/MOD08_M3.A2000214.061.2017276050502.nc',
 'MODIS/MOD08_M3.A2000245.061.2017276075932.nc',
 'MODIS/MOD08_M3.A2000275.061.2017276173030.nc',
 'MODIS/MOD08_M3.A2000306.061.2017276190346.nc',
 'MODIS/MOD08_M3.A2000336.061.2017276203640.nc',
 'MODIS/MOD08_M3.A2001001.061.2017277122349.nc',
 'MODIS/MOD08_M3.A2001032.061.2017277112301.nc',
 'MODIS/MOD08_M3.A2001060.061.2017277125646.nc',
 'MODIS/MOD08_M3.A2001091.061.2017277145429.nc',
 'MODIS/MOD08_M3.A2001121.061.2017277185847.nc',
 'MODIS/MOD08_M3.A2001152.061.2017277193228.nc',
 'MODIS/MOD08_M3.A2001182.061.2017277232025.nc',
 'MODIS/MOD08_M3.A2001213.061.2017278042002.nc',
 'MODIS/MOD08_M3.A2001244.061.2017278143507.nc',
 'MODIS/MOD08_M3.A20

We need to add a time dimension to concatenate data. For this, we define a function that will be called for each remote file (via the `preprocess` parameter of Xarray `open_mfdataset`.)

In [26]:
from datetime import datetime

In [27]:
def paths_to_datetimeindex(paths):
    return [
        datetime.strptime(date.split(".A")[-1].split(".")[0], "%Y%j") for date in paths
    ]

Xarray `open_mfdataset` allows opening multiple files at the same time.

In [28]:
# Iterate through remote_files to create a fileset
fileset = [fs.open(file) for file in remote_files]

When opening remote files, you can also select the variables you wish to analyze.

In [29]:
# Create variable used for time axis
time_var = xr.Variable("time", paths_to_datetimeindex(remote_files))
time_var

<xarray.Variable (time: 258)>
array(['2000-02-01T00:00:00.000000000', '2000-03-01T00:00:00.000000000',
       '2000-04-01T00:00:00.000000000', '2000-05-01T00:00:00.000000000',
       '2000-06-01T00:00:00.000000000', '2000-07-01T00:00:00.000000000',
       '2000-08-01T00:00:00.000000000', '2000-09-01T00:00:00.000000000',
       '2000-10-01T00:00:00.000000000', '2000-11-01T00:00:00.000000000',
       '2000-12-01T00:00:00.000000000', '2001-01-01T00:00:00.000000000',
       '2001-02-01T00:00:00.000000000', '2001-03-01T00:00:00.000000000',
       '2001-04-01T00:00:00.000000000', '2001-05-01T00:00:00.000000000',
       '2001-06-01T00:00:00.000000000', '2001-07-01T00:00:00.000000000',
       '2001-08-01T00:00:00.000000000', '2001-09-01T00:00:00.000000000',
       '2001-10-01T00:00:00.000000000', '2001-11-01T00:00:00.000000000',
       '2001-12-01T00:00:00.000000000', '2002-01-01T00:00:00.000000000',
       '2002-02-01T00:00:00.000000000', '2002-03-01T00:00:00.000000000',
       '2002-04-01T00:00:00.000000000', '2002-05-01T00:00:00.000000000',
       '2002-06-01T00:00:00.000000000', '2002-07-01T00:00:00.000000000',
       '2002-08-01T00:00:00.000000000', '2002-09-01T00:00:00.000000000',
       '2002-10-01T00:00:00.000000000', '2002-11-01T00:00:00.000000000',
       '2002-12-01T00:00:00.000000000', '2003-01-01T00:00:00.000000000',
       '2003-02-01T00:00:00.000000000', '2003-03-01T00:00:00.000000000',
       '2003-04-01T00:00:00.000000000', '2003-05-01T00:00:00.000000000',
...
       '2018-06-01T00:00:00.000000000', '2018-07-01T00:00:00.000000000',
       '2018-08-01T00:00:00.000000000', '2018-09-01T00:00:00.000000000',
       '2018-10-01T00:00:00.000000000', '2018-11-01T00:00:00.000000000',
       '2018-12-01T00:00:00.000000000', '2019-01-01T00:00:00.000000000',
       '2019-02-01T00:00:00.000000000', '2019-03-01T00:00:00.000000000',
       '2019-04-01T00:00:00.000000000', '2019-05-01T00:00:00.000000000',
       '2019-08-01T00:00:00.000000000', '2019-09-01T00:00:00.000000000',
       '2019-10-01T00:00:00.000000000', '2019-11-01T00:00:00.000000000',
       '2019-12-01T00:00:00.000000000', '2020-01-01T00:00:00.000000000',
       '2020-02-01T00:00:00.000000000', '2020-03-01T00:00:00.000000000',
       '2020-04-01T00:00:00.000000000', '2020-05-01T00:00:00.000000000',
       '2020-06-01T00:00:00.000000000', '2020-07-01T00:00:00.000000000',
       '2020-08-01T00:00:00.000000000', '2020-09-01T00:00:00.000000000',
       '2020-10-01T00:00:00.000000000', '2020-11-01T00:00:00.000000000',
       '2020-12-01T00:00:00.000000000', '2021-01-01T00:00:00.000000000',
       '2021-02-01T00:00:00.000000000', '2021-03-01T00:00:00.000000000',
       '2021-04-01T00:00:00.000000000', '2021-05-01T00:00:00.000000000',
       '2021-06-01T00:00:00.000000000', '2021-07-01T00:00:00.000000000',
       '2021-08-01T00:00:00.000000000', '2021-09-01T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [30]:
# Load in and concatenate all individual GeoTIFFs
modis = xr.concat(
    [
        xr.open_mfdataset(
            [i],
        )
        for i in fileset
    ],
    dim=time_var,
)

In [31]:
modis

<xarray.Dataset>
Dimensions:                                              (time: 258,
                                                          YDim:mod08: 180,
                                                          XDim:mod08: 360,
                                                          Effective_Optical_Depth_Average_Ocean_Micron_Levels:mod08: 7)
Coordinates:
  * time                                                 (time) datetime64[ns] ...
Dimensions without coordinates: YDim:mod08, XDim:mod08,
                                Effective_Optical_Depth_Average_Ocean_Micron_Levels:mod08
Data variables: (12/13)
    Aerosol_Optical_Depth_Land_Ocean_Mean_Mean           (time, YDim:mod08, XDim:mod08) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    AOD_550_Dark_Target_Deep_Blue_Combined_Mean_Mean     (time, YDim:mod08, XDim:mod08) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Aerosol_Optical_Depth_Average_Ocean_Mean_Mean        (time, Effective_Optical_Depth_Average_Ocean_Micron_Levels:mod08, YDim:mod08, XDim:mod08) float32 dask.array<chunksize=(1, 7, 180, 360), meta=np.ndarray>
    Cloud_Top_Temperature_Mean_Mean                      (time, YDim:mod08, XDim:mod08) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Cloud_Effective_Emissivity_Mean_Mean                 (time, YDim:mod08, XDim:mod08) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Cloud_Fraction_Mean_Mean                             (time, YDim:mod08, XDim:mod08) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    ...                                                   ...
    Cloud_Effective_Radius_Liquid_Mean_Mean              (time, YDim:mod08, XDim:mod08) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Cloud_Water_Path_Liquid_Mean_Mean                    (time, YDim:mod08, XDim:mod08) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Cloud_Water_Path_Ice_Mean_Mean                       (time, YDim:mod08, XDim:mod08) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Effective_Optical_Depth_Average_Ocean_Micron_Levels  (time, Effective_Optical_Depth_Average_Ocean_Micron_Levels:mod08) float32 dask.array<chunksize=(1, 7), meta=np.ndarray>
    XDim                                                 (time, XDim:mod08) float32 dask.array<chunksize=(1, 360), meta=np.ndarray>
    YDim                                                 (time, YDim:mod08) float32 dask.array<chunksize=(1, 180), meta=np.ndarray>
Attributes:
    HDFEOSVersion:                     HDFEOS_V2.19
    StructMetadata.0:                  GROUP=SwathStructure\nEND_GROUP=SwathS...
    StructMetadata.1:                  Min"\n\t\t\t\tDataType=DFNT_INT16\n\t\...
    StructMetadata.2:                  BJECT=DataField_295\n\t\t\tOBJECT=Data...
    StructMetadata.3:                  NT16\n\t\t\t\tDimList=("YDim","XDim")\...
    StructMetadata.4:                  \t\tDataFieldName="Cloud_Retrieval_Fra...
    StructMetadata.5:                  List=("YDim","XDim")\n\t\t\tEND_OBJECT...
    StructMetadata.6:                  CT=DataField_1037\n\t\t\t\tDataFieldNa...
    CoreMetadata.0:                    \nGROUP                  = INVENTORYME...
    ArchiveMetadata.0:                 \nGROUP                  = ARCHIVEDMET...
    identifier_product_doi:            10.5067/MODIS/MOD08_M3.061
    identifier_product_doi_authority:  http://dx.doi.org

### Apply the same pre-processing as earlier

In [32]:
modis = modis.rename_dims(
    {
        "YDim:mod08": "lat",
        "XDim:mod08": "lon",
        "Effective_Optical_Depth_Average_Ocean_Micron_Levels:mod08": "levels",
    }
)

In [33]:
modis = modis.rename_vars(
    {
        "YDim": "lat",
        "XDim": "lon",
        "Effective_Optical_Depth_Average_Ocean_Micron_Levels": "levels",
    }
)

In [34]:
modis = modis.assign_coords({"longitude": x, "latitude": y, "levels": z})

In [35]:
modis

<xarray.Dataset>
Dimensions:                                           (time: 258, lat: 180,
                                                       lon: 360, levels: 7)
Coordinates:
  * levels                                            (levels) float32 0.47 ....
    lon                                               (time, lon) float32 dask.array<chunksize=(1, 360), meta=np.ndarray>
    lat                                               (time, lat) float32 dask.array<chunksize=(1, 180), meta=np.ndarray>
  * time                                              (time) datetime64[ns] 2...
    longitude                                         (lon) float32 -179.5 .....
    latitude                                          (lat) float32 89.5 ... ...
Data variables:
    Aerosol_Optical_Depth_Land_Ocean_Mean_Mean        (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    AOD_550_Dark_Target_Deep_Blue_Combined_Mean_Mean  (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Aerosol_Optical_Depth_Average_Ocean_Mean_Mean     (time, levels, lat, lon) float32 dask.array<chunksize=(1, 7, 180, 360), meta=np.ndarray>
    Cloud_Top_Temperature_Mean_Mean                   (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Cloud_Effective_Emissivity_Mean_Mean              (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Cloud_Fraction_Mean_Mean                          (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Cloud_Optical_Thickness_Liquid_Mean_Mean          (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Cloud_Effective_Radius_Liquid_Mean_Mean           (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Cloud_Water_Path_Liquid_Mean_Mean                 (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
    Cloud_Water_Path_Ice_Mean_Mean                    (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
Attributes:
    HDFEOSVersion:                     HDFEOS_V2.19
    StructMetadata.0:                  GROUP=SwathStructure\nEND_GROUP=SwathS...
    StructMetadata.1:                  Min"\n\t\t\t\tDataType=DFNT_INT16\n\t\...
    StructMetadata.2:                  BJECT=DataField_295\n\t\t\tOBJECT=Data...
    StructMetadata.3:                  NT16\n\t\t\t\tDimList=("YDim","XDim")\...
    StructMetadata.4:                  \t\tDataFieldName="Cloud_Retrieval_Fra...
    StructMetadata.5:                  List=("YDim","XDim")\n\t\t\tEND_OBJECT...
    StructMetadata.6:                  CT=DataField_1037\n\t\t\t\tDataFieldNa...
    CoreMetadata.0:                    \nGROUP                  = INVENTORYME...
    ArchiveMetadata.0:                 \nGROUP                  = ARCHIVEDMET...
    identifier_product_doi:            10.5067/MODIS/MOD08_M3.061
    identifier_product_doi_authority:  http://dx.doi.org

:::{tip}
If you use one of xarray’s open methods such as xarray.open_dataset to load netCDF files with the default engine, it is recommended to use decode_coords=”all”. This will load the grid mapping variable into coordinates for compatibility with rioxarray. See [rioxarray documentation](https://corteva.github.io/rioxarray/stable/getting_started/getting_started.html#xarray).
:::

## Preparing and discover online datasets

With the plethora of cloud storage, there are many available online datasets. To ease the preparation and discovery of such datasets, we describe emerging community-driven initiatives promoting standards suited to both geospatial and geoscience communities. Most of the material below is adapted from a previous Pangeo 101 training {cite:ps}`galaxy2022-pangeo`.

:::{tip}
While we provide a general intro to some initiatives, we suggest below a list of FOSS4G 2022 talks with very interesting developments to prepare and discover spatio-temporal datasets in the cloud. Enjoy!

- [STAC Best Practices and Tools](https://talks.osgeo.org/foss4g-2022/talk/9RRYZM/), 2022-08-24, 11:00–11:30
- [Early use of FOSS4G in a space start up](https://talks.osgeo.org/foss4g-2022/talk/HG7RLR/), 2022-08-24, 11:30–12:00
- [Exploring Data Interoperability with STAC and the Microsoft Planetary Computer](https://talks.osgeo.org/foss4g-2022/talk/L3KNY8/), 2022-08-24, 12:10–12:15
- [Serving oblique aerial imagery using STAC and Cloud Optimized Geotiffs](https://talks.osgeo.org/foss4g-2022/talk/SQYE9A/), 2022-08-24, 14:45–15:15
- [Pangeo Forge: Crowdsourcing Open Data in the Cloud](https://talks.osgeo.org/foss4g-2022/talk/DABTGG/). 2022-08-26, 10:00-10:30.
:::

### Analysis Ready, cloud optimized data (ARCO)
When analyzing data at scale, the data format used is key. For years, the main data format was netCDF e.g. Network Common Data Form but with the use of cloud computing and interest in Open Science, different formats are often more suitable.

Formats for analyzing data from the cloud are refered to as "Analysis Ready, Cloud Optimized" data formats or in short ARCO. Find further info about ARCO datasets in {cite:ps}`Abernathey2022-arco`.

What is "Analysis Ready"?
* Think in terms of "Datasets" not "data files"
* No need for tedious homogenizing / cleaning setup guides
* Curated and cataloged

What is "Cloud Optimized"?
* Compatible with object storage e.g. access via HTTP
* Supports lazy access and intelligent subsetting
* Integrates with high-level analysis libraries and distributed frameworks

Instead of having a big dataset, ARCO datasets are chunked appropriately for analysis and have rich metadata (See Figure 1).

<img src="https://github.com/galaxyproject/training-material/blob/696dfecd4c88e59b487a7a3557cfedca6ec5754b/topics/climate/images/arco_data.png?raw=true" align="Left" /></a>

*Fig 1. Example of an ARCO dataset. Source: {cite:ps}`galaxy2022-pangeo`.*

### The Pangeo forge initiative

[Pangeo Forge](https://pangeo-forge.org/) is an open source platform for data **Extraction**, **Transformation**, and **Loading** (ETL). The goal of *Pangeo Forge* is to make it easy to extract data from traditional repositories and deposit this data in cloud object storage in an analysis-ready, cloud optimized (ARCO) format {cite:ps}`galaxy2022-pangeo`.

Pangeo Forge is inspired directly by Conda Forge, a community-led collection of recipes for building conda packages.

It is under active development and the Pangeo community hopes it will play a role in democratizing the publication of datasets in ARCO format.

#### How does Pangeo Forge work?

Pangeo Forge defines the concept of a recipe, which specifies the logic for transforming a specific data archive into an ARCO data store.
All contributions to Pangeo Forge must include an executable Python module, named recipe.py or similar, in which the data transformation logic is embedded (Figure 2).
The recipe contributor is expected to use one of a predefined set of template algorithms defined by Pangeo Forge.
Each of these templated algorithms is designed to transform data of a particular source type into a corresponding ARCO format, and requires only that the contributor populate the template with information unique to their specific data transformation, including the location of the source files and the way in which they should be aligned in the resulting ARCO data store {cite:ps}`Abernathey2022-arco`.

The diagram below looks complicated but like for conda forge most of the process is automated.

<img src="https://www.frontiersin.org/files/Articles/782909/fclim-03-782909-HTML/image_m/fclim-03-782909-g002.jpg" align="Left" /></a>

*Fig 2. A recipe in relation to Pangeo Forge architecture. Source: {cite:ps}`Abernathey2022-arco`.*

The next step after preparing the dataset is then to tell the community where and how to access to your transformed dataset.

This is done by creating a catalog.

### Spatio Temporal Asset Catalogs (STAC)

The [STAC](https://stacspec.org/en/) specification is a common language to describe geospatial information, so it can more easily be worked with, indexed, and discovered.

#### Why STAC?
* Each provider has its own catalog and interface (APIs).
* Every time you want to access a new catalog, you need to change your program.
* We have lots of data providers and each with a bespoke interface.
* It is becoming quickly difficult for programmers who need to design a new data connector each time.

#### Features
- STAC catalogs are extremely simple.
- They are composed of three layers:
    - **Catalogs**
        - **Collections**
            - **Items**
- STAC is very popular for Earth Observation satellite imagery.
- For instance it can be used to access Sentinel-2 in AWS (see Figure 3).

<img src="https://github.com/galaxyproject/training-material/blob/696dfecd4c88e59b487a7a3557cfedca6ec5754b/topics/climate/images/sentinel2_AWS.png?raw=true" align="Left" /></a>

*Fig 3. Example of STAC collection of Sentinel-2 images hosted in AWS.
Source: {cite:ps}`galaxy2022-pangeo`.*


#### STAC and Pangeo Forge
- Pangeo-forge supports the creation of analysis-ready cloud optimized (ARCO) data in cloud object storage from "classical" data repositories.
- STAC is used to create catalogs and goes beyond the Pangeo ecosystem.
- Work is ongoing to figure out the best way to expose Pangeo-Forge-generated data assets via STAC catalogs.

:::{tip}
Pangeo members, Scott Henderson (University of Washington) and Tom Augspurger (Microsoft), provided a great workshop in FOSS4G 2021 covering STAC.

Feel free to explore the GitHub repository of the  [here](https://github.com/pangeo-data/foss4g-2021).
:::

<div class="alert alert-success">
    <i class="fa-check-circle fa" style="font-size: 22px;color:#666;"></i> <b>Key Points</b>
    <br>
    <ul>
        <li>Access to remote dataset</li>
        <li>ARCO datasets</li>
        <li>Pangeo Forge</li>
        <li>STAC</li>
    </ul>
</div>

## References

```{bibliography}
:style: alpha
:filter: topic % "data-access"
```

## Packages citation

```{bibliography}
:style: alpha
:filter: topic % "access" and topic % "package"
:keyprefix: c-
```